In [ ]:
# Combine spec int 2006 and 2017, filter TPC-H CPUs.

In [ ]:
%pip install pandas duckdb matplotlib seaborn

import pandas as pd
import duckdb
import os
import matplotlib.pyplot as plt
import seaborn as sns

key_vendor = "Hardware Vendor\t"
key_system = "System"
# SpecRate: "Processor", SpecSpeed: "Processor "
key_processor = "Processor "

file_combined_path = '../spec_speed/spec-cint2006-2017.csv'

# Avoid data cropping when using display()
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

if False and os.path.exists(file_combined_path):
    df = pd.read_csv(file_combined_path)
else:
    # Load data from the first CSV file
    file1_path = '../spec_speed/spec-cint2006-results-20240730-120726.csv'
    if os.path.exists(file1_path):
        data1 = pd.read_csv(file1_path)
    else:
        raise FileNotFoundError(f"The file {file1_path} does not exist.")

    # Load data from the second CSV file
    file2_path = '../spec_speed/spec-cint2017-results-20240730-120816.csv'
    if os.path.exists(file2_path):
        data2 = pd.read_csv(file2_path)
    else:
        raise FileNotFoundError(f"The file {file2_path} does not exist.")

    # Add a column to each dataframe to label the source
    data1['Source'] = 'cint2006'
    data2['Source'] = 'cint2017'
    # 2006 columns: 'Result' 'Baseline'
    # 2017 columns: 'Result' 'Baseline'
    data1['perf'] = data1['Result']
    data2['perf'] = data2['Result'] * 9
    # data2["perf"] = data2["Result"]

    # Concatenate the dataframes
    df = pd.concat([data1, data2], ignore_index=True)
    # df = data2
    df = df[df["HW Avail"].notna()]
    df["hw_avail_year"] = df["HW Avail"].str.split('-').str[1].astype(int)
    df["perf_per_chip"] = df["perf"] / df["# Chips"]
    # Filter out 0 values
    df = df[df["perf"] != 0]
    # df["baseline_per_core"] = df["Baseline"] / df["# Cores"]
    # Use below line to write data
    df.to_csv(file_combined_path, index=False)
display(df.columns)
display(df.index)
    
    
# con = duckdb.connect("../spec.db")
# con.sql("DROP TABLE IF EXISTS spec")
# con.sql("CREATE TABLE spec AS SELECT * FROM df")
# con.close()


In [ ]:
shared_df = pd.merge(data1, data2, on=['Processor ', 'Memory', '# Cores', '# Chips'], suffixes=['_2006', '_2017'])
shared_df = shared_df[(shared_df['Result_2006'] > 0) & (shared_df['Result_2017'] > 0)]
factor_df = shared_df['Result_2006'] / shared_df['Result_2017']
print(shared_df['Result_2006'].describe())
print()
print(shared_df['Result_2017'].describe())
print()
print(factor_df.describe())

In [ ]:
plt.close('all')
# display(df)
plt.figure(figsize=(4, 2))
# y_val = 'Baseline'
y_val = 'perf_per_chip'
#y_val = 'baseline_per_core'
plt.scatter(
    df['hw_avail_year'],
    df[y_val],
    marker='x')
plt.xlabel('Year')
plt.ylabel(y_val)
plt.title('Performance over Years')
plt.xticks(ticks=df['hw_avail_year'].unique(), rotation=90)
plt.grid(True)
plt.show()
plt.close()

In [ ]:
# Count the occurrences of each distinct value in the "key_vendor" column
value_counts = df[key_vendor].value_counts()

# Create a histogram
plt.figure(figsize=(10, 2))
value_counts.plot(kind='bar')
plt.title('Occurrences of Distinct Values in key_vendor Column')
plt.xlabel('key_vendor')
plt.ylabel('Occurrences')
plt.xticks(rotation=90)
plt.show()

In [ ]:
import re

# Dynamic approach
df_intel_tpc = pd.read_csv("../tpc_xeon_w_launch_years.csv")

def crop_model_name(name):
    # Create a regex pattern that matches any keyword
    pattern = "|".join(["Intel Xeon", "CPU", "Bronze", "Gold", "Silver", "Platinum", "E5-", "E7-"])
    # Use regex to remove all keywords in one go
    name = re.sub(pattern, "", name)
    # if len(name) > 1 and not name[-1].isdigit():
    #     name = name[:-1]
    return name.strip()

# print(df_intel_tpc['model'].unique())
df_intel_tpc['short_model'] = df_intel_tpc['model'].apply(crop_model_name)
# Remove empty ones, i.e., Intel Xeon - unclear what model was used.
df_intel_tpc = df_intel_tpc[df_intel_tpc['short_model'] != ""]
long_intel_tpc_models = df_intel_tpc['model'].unique()
intel_tpc_models = df_intel_tpc['short_model'].unique()
print(long_intel_tpc_models)
print(intel_tpc_models)
# print(len(intel_tpc_models))

# only consider records with one socket
# df = df[(df["# Chips"] == 1)]
df["short_model"] = df[key_processor].apply(lambda x: next((model for model in intel_tpc_models if model in x), None))
df = df[df["short_model"].notna()]
# Filter out rows where 'Processor' contains "v" but 'short_model' does not
df = df[~((df[key_processor].str.contains('v')) & (~df['short_model'].str.contains('v')))]
# print(df.columns)
columns = ['Benchmark', 'Hardware Vendor\t', 'System', '# Cores', '# Chips',
       '# Cores Per Chip', '# Threads Per Core', 'Processor ', 'Processor MHz',
       'Processor Characteristics', 'CPU(s) Orderable', 'Auto Parallelization',
       'Base Pointer Size', 'Peak Pointer Size', 'HW Avail', 'SW Avail',
       'Result', 'Baseline',
       'Tested By', 'Test Date', 'Published', 'Updated ', 'Source', 'perf',
       'hw_avail_year', 'perf_per_chip',
       'short_model']
# reduce columns
df = df[columns]
sources = df["Source"].unique()
# print(df_reduced["short_model"].unique())
# print(len(df_reduced["short_model"].unique()))
# print(df_reduced["# Chips"].unique())



# model_selection = ['Gallatin', '5160', 'X5355',
#  '7040', '7041', '7140', '7140',
#  '7150', 'E5320', 'E5345',
#  'X5365', 'X5460', 'X7350',
#  'E5420', 'X5440', 'X7460',
#  'E5520', 'X5570', 'X5680',
#  'X5650', 'X7560', 'X5690',
#  '8870', '2690', '4650',
#  '2680', '8890', '8891',
#  '2690', '2699', '2680',
#  '4890', '2670', '2699',
#  '8180', '2630',
#  '4114', '6150', '6148',
#  '8163', '4110',
#  '4210', '6258',
#  '6354', '8255',
#  '4410']

In [ ]:
# Group by key_processor and aggregate the metric "perf_per_chip"
grouped = df.groupby([key_processor, 'short_model', 'Source'])['perf_per_chip'].agg(['mean', 'std', 'median']).reset_index()
# display(grouped)

# Renaming columns for better understanding
grouped = grouped.rename(columns={"mean":'Average Perf Per Chip', 'std':'Std Dev Perf Per Chip',"median":'Median Perf Per Chip'})
# grouped.columns = ['Processor', 'Average Perf Per Chip', 'Std Dev Perf Per Chip', 'Median Perf Per Chip']
# display(grouped)

# Melting the DataFrame for easier plotting
grouped_melted = grouped.melt(id_vars=[key_processor, 'Std Dev Perf Per Chip','short_model','Source'], 
                              value_vars=['Average Perf Per Chip', 'Median Perf Per Chip'],
                              var_name='Metric', value_name='Perf Per Chip')
# display(grouped_melted)

# Plotting using seaborn
plt.figure(figsize=(10, 1.5))
bar_plot = sns.barplot(x=key_processor, y='Perf Per Chip', hue='Metric', data=grouped_melted, errorbar=None)
for i in range(len(grouped)):
    bar_plot.errorbar(x=i-0.2, y=grouped.iloc[i]['Average Perf Per Chip'],
                      yerr=grouped.iloc[i]['Std Dev Perf Per Chip'], fmt='none', c='red', capsize=5)

plt.xlabel('Processor')
plt.ylabel('Perf Per Chip')
plt.title('Average and Median Perf Per Chip with Std Dev by Processor')
plt.xticks(rotation=90)
plt.legend(title='Metric')
plt.grid(axis='x')  # Add this line to enable x-axis grid lines
plt.show()


In [ ]:
median_spec_perf = grouped_melted[grouped_melted["Metric"].str.contains("Median")]
median_spec_perf = median_spec_perf.drop(columns=["Std Dev Perf Per Chip", "Metric"])
median_spec_perf.rename(columns={"Perf Per Chip": "median_spec_int_speed_perf"}, inplace=True)

# display(median_spec_perf)

df_combined = median_spec_perf.merge(df_intel_tpc, on="short_model", how="left")

# display(df_combined)

df_combined.to_csv("../spec_speed/spec-median-{}-tpc-cpus.csv".format("".join(sources)),header=True)
# display(median_spec_perf)

In [ ]:

# df = pd.read_csv('../tpc_xeon_perf_spec2006-2017combined.csv')

# merged_df = pd.merge(df, median_spec_perf, left_on='model', right_on='Processor', how='left')

# # Display the merged DataFrame
# print(merged_df)
